In [17]:
using LinearAlgebra


# decompose A into LU, with partial pivoting
function LU_decomp(A::Array{Float64,2})
    n = size(A,1)

    # identity matrix
    id = [1. for _ in 1:n]

    # list for storing transformation matrices
    tx_mats = []

    # perform partial pivoting on A first
    pivot!(A)

    for i in 1:n
        # initialize the transformation matrix
        tx = diagm(id)
        
        # for each row below a pivot, record the "pivoting scalar" and place
        # it in the corresponding position in the transformation matrix
        for j in i+1:n
            tx[j,i] = -A[j,i]/A[i,i]
        end
        
        # perform the transformation
        A = tx*A
        
        # record the transformation matrix
        push!(tx_mats, tx)
    end
    
    # return U and L where L is the inverse of
    # the product of the transformation matrices
    A, inv(reduce(*, reverse(tx_mats)))
end


# Gaussian elimination
function ref!(A::Array{Float64,2})
    rows = size(A,1)
    pivot!(A)

    # zero out the entries in the rows below each pivot
    for i in 1:rows
        for j in i+1:rows
            # "pivoting scalar" which will cancel the entries below the pivot
            p = -A[j,i]/A[i,i]
            A[j,:] += p*A[i,:]
        end
    end
end


# Perform partial pivoting on matrix A
function pivot!(A::Array{Float64,2})
    # search for the largest absolute value in each pivot column
    # row swap so that this value is in the pivot position
    for col in 1:size(A,1)-1
        swap!(A, col, (col-1) + argmax(abs.(A[col:end, col])))
    end

    # check if row reduction will result in
    # a zero on the diagonal, repivot if needed
    for i in 2:size(A,1)
        if A[i,i] == A[i-1,i]
            swap!(A, i, i+1)
        end
    end
end


# swap rows i and j of matrix A
function swap!(A::Array{Float64,2}, i::Int, j::Int)
    # swap two rows element-wise (faster than slicing)
    for k in 1:size(A,2)
        A[i,k], A[j,k] = A[j,k], A[i,k]
    end
end


# solve a system using backward substitution
function backward_sub(A, b)
    num_rows = size(A,1)
    
    # initializes an empty solution vector
    xx = zeros(num_rows)
    for i in num_rows:-1:1
        # solves for each variable using the
        # previously solved variables on which it depends
        xi = mapreduce(j -> A[i,j]*xx[j], +, i+1:num_rows, init=0)
        xx[i] = (b[i]-xi)/A[i,i]
    end
    return xx
end


# solve a system using forward substitution
function forward_sub(A, b)
    num_rows = size(A,1)
    
    # initializes an empty solution vector
    xx = zeros(num_rows)
    for i in 1:num_rows
        # solves for each variable using the
        # previously solved variables on which it depends
        xi = mapreduce(j -> A[i,j]*xx[j], +, 1:i-1, init=0)
        xx[i] = (b[i]-xi)/A[i,i]
    end
    return xx
end


# Perform gaussian elimination and solve for b using backward substitution
function gauss_solve(A, b)
    # augment A with b
    A = hcat(A,b)
    
    # reduce to echelon form
    ref!(A)
    
    # split A and b again
    A, b = A[:,1:end-1], A[:, end]

    return backward_sub(A, b)
end


# compare gaussian elimination vs LU decomposition
# vs Julia's built-in matrix methods
function compare(A, b)
    LU_solve(A, b)
    gauss_solve(A, b)
    A\b
    
    println("LU:       ", round.(LU_solve(A, b), digits=8))
    @time LU_solve(A, b)

    println("\ngauss:    ", round.(gauss_solve(A, b), digits=8))
    @time gauss_solve(A, b)

    println("\nbuilt-in: ", round.(A\b, digits=8))
    @time A\b
    
    println("\n\n")
end


# decompose A into LU and solve for b
function LU_solve(A, b)
    U, L = LU_decomp(A)
    return backward_sub(U, forward_sub(L, b))
end;

In [18]:
A = [5. 6. 7. 8.; 0. 4. 3. 2.; 0. 0. 0. 1.; 0. 0. -1. -2.]
b = [26, 9, 1, -3]
compare(A,b)

A = [2. -1. 1.; 3. 3. 9.; 3. 3. 5.]
b = [-1., 0., 4.]
compare(A,b)

A = [1. -1. 0; 2. 2. 3.; -1. 3. 2.]
b = [2., -1., 4.]
compare(A,b)

A = [2.1756 4.0231 -2.1732 5.1967;
    -4.0231 6.0 0.0 1.1973;
    -1. -5.2107 1.1111 0.;
    6.0235 7. 0. -4.1561]
b = [17.102, -6.1593, 3.0004, 0.]
compare(A,b)

A = [2. 1. 0. 0.; -1. 3. 3. 0.; 2. -2. 1. 4.; -2. 2. 2. 5.]
b = [0., 5., -2., 6.]
compare(A,b)

LU:       [3.0, 0.0, 5.0, -3.0]
  0.000024 seconds (37 allocations: 4.031 KiB)

gauss:    [3.0, 0.0, 5.0, -3.0]
  0.000009 seconds (39 allocations: 4.438 KiB)

built-in: [3.0, 0.0, 5.0, -3.0]
  0.000001 seconds (1 allocation: 112 bytes)



LU:       [1.55555556, 1.86111111, -1.25]
  0.000010 seconds (29 allocations: 2.688 KiB)

gauss:    [1.55555556, 1.86111111, -1.25]
  0.000003 seconds (24 allocations: 2.391 KiB)

built-in: [1.55555556, 1.86111111, -1.25]
  0.000002 seconds (4 allocations: 416 bytes)



LU:       [-6.5, -10.5, 12.0]
  0.000007 seconds (29 allocations: 2.688 KiB)

gauss:    [-6.5, -10.5, 12.0]
  0.000003 seconds (24 allocations: 2.391 KiB)

built-in: [-6.5, -10.5, 12.0]
  0.000011 seconds (4 allocations: 416 bytes)



LU:       [2.91932402, 0.67939446, 5.8135584, 1.26039062]
  0.000008 seconds (37 allocations: 4.031 KiB)

gauss:    [2.91932402, 0.67939446, 5.8135584, 1.26039062]
  0.000004 seconds (39 allocations: 4.438 KiB)

built-in: [2.91932402, 0.67939446, 5.81355